In [26]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged_novo

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob
import shutil


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

### Get Urls

In [275]:
download_dict = caged_novo.get_download_links()

estabelecimentos
movimentacoes


### DOWNLOAD E TRATAMENTO

- download
- extrair
- load
- padroniza nome das colunas
- limpa
- salva, formato ano=xxxx/mes=xx/sigla_uf=XX
- upload bd
- deleta arquivos

In [297]:
RAW_PATH = "../data/caged_novo/raw/"
CLEAN_PATH = "../data/caged_novo/clean/"

dd = pd.DataFrame()
#deleta pasta
shutil.rmtree(CLEAN_PATH)
shutil.rmtree(RAW_PATH)
for tipo in list(download_dict.keys())[1:]:
    
    download_opt = download_dict[tipo]["check_download"]
    download_opt.update(download_dict[tipo]["must_download"])
    # TODO: etapa de filtragem do que n deve ser baixado
    
    for year_month_path in list(download_opt.keys()):
        print(tipo,": ",year_month_path)
        ## download data
        save_path = RAW_PATH + f"{tipo}/" + year_month_path
        caged_novo.download_data(save_path, download_opt[year_month_path])
        
        #create some vars
        file_name = download_opt[year_month_path].split("/")[-1].split(".")[0]
        file_path = RAW_PATH + f"{tipo}/" + year_month_path
        
        #extrai arquivo
        caged_novo.extract_file(file_path, file_name, save_rows=None)

        # load e organiza os dados
        df = caged_novo.rename_add_orginaze_columns(file_path, file_name, tipo)

        # salva no formato de particao
        save_clean_path = CLEAN_PATH + f"{tipo}/"
        caged_novo.creat_partition(df,save_clean_path,year_month_path)
        
        #upload basedosdados
        caged_novo.upload_to_bd(tipo,save_clean_path)
        
        #deleta pasta
        shutil.rmtree(CLEAN_PATH + f"{tipo}/")
        shutil.rmtree(RAW_PATH + f"{tipo}/")
    
#     print('\n')


Get download links
estabelecimentos
movimentacoes


estabelecimentos
nothing to download


movimentacoes
nothing to download




Downloading: 100%|██████████████████████| 5570/5570 [00:00<00:00, 9060.92rows/s]


,sigla_uf,id_municipio,id_municipio_6
0,RO,1100015,110001
1,RO,1100023,110002
2,RO,1100031,110003
3,RO,1100049,110004
4,RO,1100056,110005
...,...,...,...
5565,GO,5222005,522200
5566,GO,5222054,522205
5567,GO,5222203,522220
5568,GO,5222302,522230


### Create table and fill metadata

In [53]:
tb = bd.Table('microdados_movimentacoes','br_me_caged')
st = bd.Storage('br_me_caged','microdados_movimentacoes',)
st.delete_table(bucket_name='basedosdados-dev')
# tb.create(path='../data/caged_novo/clean/movimentacoes/',
#           if_table_config_exists='pass',
#           if_storage_data_exists='replace',
#           if_table_exists='replace'
#          )

# tb.update_columns('https://docs.google.com/spreadsheets/d/1ihyOCSkaarmR3uMHj8bmP9tv-OChVeL2LkEWaGqxp7w/edit#gid=787251136')

# tb.publish(if_exists='replace')

In [54]:
tb = bd.Table('microdados_estabelecimentos','br_me_caged')
st = bd.Storage('br_me_caged','microdados_estabelecimentos',)
st.delete_table(bucket_name='basedosdados-dev')
# tb.create(path='../data/caged_novo/clean/estabelecimentos/', 
#           if_table_config_exists='pass',
#           if_storage_data_exists='replace',
#           if_table_exists='replace' 
#          )

# tb.update_columns('https://docs.google.com/spreadsheets/d/1-XQM9mhlJ3-t10c7cPpwOOK24hKz0u3qR6m04I1PoR4/edit#gid=1885260190')

# tb.publish(if_exists='replace')

In [55]:
## fill columns description


In [46]:
### SQL